# Chi-Squared Tests



# Chi-Squared Goodness-Of-Fit Test

In [213]:
import numpy as np
import pandas as pd
import scipy.stats as stats

In [214]:


national = pd.DataFrame(["white"]*100000 + ["hispanic"]*60000 +\
                        ["black"]*50000 + ["asian"]*15000 + ["other"]*35000)
           

minnesota = pd.DataFrame(["white"]*600 + ["hispanic"]*300 + \
                         ["black"]*250 +["asian"]*75 + ["other"]*150)

national_table = pd.crosstab(index= national[0], columns="count")
minnesota_table = pd.crosstab(index= minnesota[0], columns="count")

print( "National")
print(national_table)
print(" ")
print( "Minnesota")
print(minnesota_table)


Calculamos a formula do chi_squared por:

chi_squared = sum(((observed - expected)**2)/expected).

Neste caso, os dados observados são os dados de minnesota e os dados experados é obtido pelo produto da proporção de cada variável categórica com o número de elementos de minnesota. 

In [215]:
observed = minnesota_table

national_ratios = national_table/len(national)  
expected = national_ratios * len(minnesota)  

chi_squared_stat = (((observed-expected)**2)/expected).sum()

print(chi_squared_stat)

Similar ao teste-t onde comparamos com um valor crítico baseado a distribuição-t, aqui comparamos com um valor crítico obtido apartir da distribuição-chi_squared. Vamos encontrar o valor crítico para C.L 95% e checar o p_value para comparar com nosso resultar.

In [216]:
crit = stats.chi2.ppf(q = 0.95,
                     df = 4)
p_value = 1 - stats.chi2.cdf(x = chi_squared_stat,
                            df = 4)

print("Critical Value")
print(crit)
print("\n")
print("p_value")
print(p_value)

Como o valor valor do chi_quadrado obtido excede o valor crítico, então rejeitamos a hipótese nula, e portanto, os dandos são os mesmos.

Podemos fazer o teste do chi-squared automaticamente, onde tem como padrão C.L de 95%.

In [217]:
stats.chisquare(f_obs = observed,
               f_exp = expected)

# Chi-Squared Test of Independence

In [218]:
np.random.seed(10)
voter_race = np.random.choice(a= ["asian","black","hispanic","other","white"],
                              p = [0.05, 0.15 ,0.25, 0.05, 0.5],
                              size=1000)

voter_party = np.random.choice(a= ["democrat","independent","republican"],
                              p = [0.4, 0.2, 0.4],
                              size=1000)

voters = pd.DataFrame({"race":voter_race, 
                       "party":voter_party})

voter_tab = pd.crosstab(voters.race, voters.party, margins = True)

voter_tab.columns = ["democrat","independent","republican","row_totals"]

voter_tab.index = ["asian","black","hispanic","other","white","col_totals"]

observed = voter_tab.iloc[0:5,0:3]   
voter_tab


*Note que o voter_race e voter_party  são gerados independentemente.*

Para um teste de independencia usamos a mesma formula para o chi_squared usando no teste goodness-fit. 

A diferença principal do caso feito anterior best-fit é que precisamos calcular o número esperado parada cada célula em uma tabela 2-d no caso anterior era em uma tabela 1-d. Para fazer o calculo, basta tomarmano o produto do número total de um etinia com o número total de uma classe de votantes e dividir pelo número total de votantes, e.g., a estimativa do número de asiaticos democratas é calculada por:

(número total de asiáticos x número total de decocratas)/1000 = (60 x 397)/1000 = 23.820.

Podemos fazer este cálculo de forma pratica usando a função np.outer()

In [219]:
expected =  np.outer(voter_tab["row_totals"][0:5],
                     voter_tab.loc["col_totals"][0:3]) / 1000

expected = pd.DataFrame(expected)

expected.columns = ["democrat","independent","republican"]
expected.index = ["asian","black","hispanic","other","white"]

expected

Agora o valor do chi-squared pode ser calculado análogo ao caso anterior. Contudo, precisamos performar uma soma duas vezes, uma primeira que estará somando sobre toda uma coluna formando uma matriz linha e depois outra que soma sobre essa matriz linha, resultando em um número. 

In [220]:
chi_squared_stat = (((observed-expected)**2)/expected).sum().sum()

print(chi_squared_stat)

Calculando o valor crítico a C.L 95% e considerando que os grais de liberdade é dado pelo produto do nº de cada categoria menos um, ou seja, temos 5 categorias de etnias e 3 categorias de identidade política, removemos uma unidade da categoria de etnia e uma unidade da categoria identidade política, assim teremos que df = 4x2 = 8.

In [228]:
crit = stats.chi2.ppf(q = 0.95,
                     df = 8)

print("critical value:")
print(crit)

p_value = 1 - stats.chi2.cdf(x = chi_squared_stat,
                        df = 8)

print("p value:")
print(p_value)

Da mesma forma no teste chi-squared goodness-fit podemos fazer de forma automática pela função *stas.chi2_contingency()*. 

Nos retornando o valor do chi-squared, o p-value, e nº de graus de liberdade.

In [227]:
stats.chi2_contingency(observed = observed)

Podemos analiar pelo valor crítico, sendo este maior que o valor do chi-squared aceitamos a hipótese nula e portanto não há nada relacionando os dois dados, ou ainda, devido ao auto valor do p-value aceitamos a hipótese nula, obtendo assim o mesmo resultado.